In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/labeled.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170320 entries, 0 to 170319
Data columns (total 56 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          170320 non-null  object 
 1   timestamp     170320 non-null  int64  
 2   ask           170320 non-null  float64
 3   bid           170320 non-null  float64
 4   askvol        170320 non-null  float64
 5   bidvol        170320 non-null  float64
 6   ask10         170320 non-null  float64
 7   ask20         170320 non-null  float64
 8   ask30         170320 non-null  float64
 9   ask40         170320 non-null  float64
 10  ask50         170320 non-null  float64
 11  ask60         170320 non-null  float64
 12  ask70         170320 non-null  float64
 13  ask80         170320 non-null  float64
 14  ask90         170320 non-null  float64
 15  ask100        170320 non-null  float64
 16  ask110        170320 non-null  float64
 17  ask120        170320 non-null  float64
 18  ask1

In [4]:
sum(data['position'] == -1), sum(data['position'] == 1)

(212, 227)

In [5]:
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression

In [6]:
rfc = IsolationForest()
data['change'] = data.ask.pct_change()

In [7]:
sum(data['position'])

15.0

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['position', 'date', 'timestamp', 'days'], axis=1).dropna(), data.drop(['date', 'timestamp', 'days'], axis=1).dropna()['position'], test_size=0.4)

In [10]:
y_test

107383    0.0
141792    0.0
159971    0.0
19707     0.0
120896    0.0
         ... 
86388     0.0
65178     0.0
137239    0.0
38816     0.0
66228     0.0
Name: position, Length: 68081, dtype: float64

In [11]:
lin = LinearRegression()

In [12]:
lin.fit(X_train, y_train)

LinearRegression()

In [13]:
pred = lin.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score, mean_absolute_error, roc_auc_score

In [15]:
sum(pred)

6.408020103863865

# Drop one class in each model

Convert the experiment to a single class anomaly detection problem.

In [60]:
buy_signals = data.drop(data[data.position < 0].index)


In [61]:
buy_signals.describe()

timestamp            ask            bid         askvol  \
count  1.701080e+05  170108.000000  170108.000000  170108.000000   
mean   1.614520e+12   45754.865938   45760.702010      38.145850   
std    4.918908e+07    1232.198015    1231.203884      54.665071   
min    1.614435e+12   43000.010000   43004.800000       0.000000   
25%    1.614477e+12   44802.930000   44811.642500       6.652615   
50%    1.614520e+12   46023.470000   46029.300000      13.773853   
75%    1.614563e+12   46782.375000   46785.210000      43.368536   
max    1.614605e+12   48439.930000   48440.810000    1865.787806   

              bidvol          ask10          ask20          ask30  \
count  170108.000000  170108.000000  170108.000000  170108.000000   
mean      194.513363       1.649342       1.315434       1.234824   
std       189.104561       3.470342       2.029377       2.006460   
min         0.011218       0.000000       0.000000       0.000000   
25%        74.744326       0.134608       0.356538       0.202389   
50%       137.646214       0.768750       0.852620       0.786000   
75%       260.650054       2.142636       1.721281       1.751069   
max      2630.646596     134.897135     138.382347     166.934472   

               ask40          ask50  ...         bid180         bid190  \
count  170108.000000  170108.000000  ...  170108.000000  170108.000000   
mean        1.016127       0.649814  ...       0.218578       0.177853   
std         2.058862       1.682717  ...       1.228660       1.185845   
min         0.000000       0.000000  ...       0.000000       0.000000   
25%         0.003330       0.000000  ...       0.000000       0.000000   
50%         0.464287       0.099474  ...       0.000000       0.000000   
75%         1.459935       0.917317  ...       0.000000       0.000000   
max       166.793277     165.817030  ...     151.731743     155.873628   

              bid200            mid        change       askvolch  \
count  170108.000000  170108.000000  1.701070e+05  170108.000000   
mean        0.144771   45757.783974 -7.535895e-08       5.220032   
std         1.053835    1231.695539  1.796488e-04       5.235838   
min         0.000000   43002.485000 -4.869209e-03       0.000056   
25%         0.000000   44807.363750 -5.515551e-05       3.004654   
50%         0.000000   46026.017500  0.000000e+00       4.064210   
75%         0.000000   46783.835000  5.979592e-05       4.749247   
max       151.855850   48439.955000  1.058950e-02     247.187780   

            bidvolch          midch      trendline       position  
count  170108.000000  170108.000000  169989.000000  170108.000000  
mean       10.012523      30.359152   45756.498157       0.001334  
std         9.426748      17.608039    1230.300224       0.036506  
min         0.000113       0.000119   43118.592375       0.000000  
25%         4.178689      15.869545   44806.291208       0.000000  
50%         6.094745      28.274263   46026.775333       0.000000  
75%        13.621794      41.661586   46785.915417       0.000000  
max       177.159286      78.517964   48375.619250       1.000000  

[8 rows x 52 columns]

In [44]:
len(buy_signals)

170108

In [19]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()

In [151]:
X_train, X_test, y_train, y_test = train_test_split(buy_signals.drop(['position', 'date', 'timestamp', 'days'], axis=1).dropna(), buy_signals.drop(['date', 'timestamp', 'days'], axis=1).dropna()['position'], test_size=0.4)
X = buy_signals.drop(['position', 'date', 'timestamp', 'days'], axis=1).dropna()
y = buy_signals.drop(['date', 'timestamp', 'days'], axis=1).dropna()['position']

In [120]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
roc_scorer = make_scorer(roc_auc_score)
lin = LinearRegression()

In [121]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(lin, X, y, scoring=roc_scorer, cv=cv, n_jobs=-1)

In [122]:
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.803 (0.051)


In [123]:
n_scores

array([0.85095075, 0.79463124, 0.7642503 , 0.79927929, 0.79541706,
       0.77785006, 0.77382904, 0.81645443, 0.74791522, 0.92119623,
       0.75022089, 0.84035085, 0.81075073, 0.7032204 , 0.75491231,
       0.82109269, 0.83965086, 0.80934977, 0.87910298, 0.80677802,
       0.8811092 , 0.8434513 , 0.81996066, 0.7208335 , 0.85303549,
       0.76695232, 0.76373038, 0.71405411, 0.79689741, 0.85894525])

In [124]:
sell_signals = data.drop(data[data.position > 0].index)


In [128]:
X = sell_signals.drop(['position', 'date', 'timestamp', 'days'], axis=1).dropna()
y = sell_signals.drop(['date', 'timestamp', 'days'], axis=1).dropna()['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


In [129]:
sell_signals.position.sum()

-212.0

In [130]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(lin, X, y, scoring=roc_scorer, cv=cv, n_jobs=-1)

In [131]:
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.850 (0.045)


In [152]:
lin.fit(X_train, y_train)

LinearRegression()

In [153]:
y_pred = lin.predict(X_train) 

In [154]:
sum(y_train) / len(y_train)

0.0013040110595825204

In [156]:
threshold = 0.015
sum(y_pred >= threshold) / len(y_pred)

0.0007745629602031512

In [157]:
sum(y_pred >= threshold)

79

In [158]:
roc_auc_score(y_train, y_pred >= threshold)

0.5146694416189699

In [160]:
y_pred = lin.predict(X_test)
sum(y_pred >= threshold) / len(y_pred)

0.0009706453320783576

In [161]:
roc_auc_score(y_test, y_pred >= threshold)

0.5101670304575366

In [162]:
roc_curve(y_test, y_pred >= threshold)

(array([0.0000000e+00, 9.4253483e-04, 1.0000000e+00]),
 array([0.       , 0.0212766, 1.       ]),
 array([2, 1, 0]))

In [165]:
import numpy as np
roc_auc_score(y_test, np.zeros(len(y_pred)))

0.5